Возьмите видео с ютуб, аудио-файл и текстовый документ по ссылкам, транскрибируйте видео и аудио -дорожку, полученные тексты соедините с текстовым документом, на основе получившегося текста создайте нейро-консультанта для ответов на вопросы по данным материалам, а также краткую методичку.

Видео: https://www.youtube.com/watch?v=LRuszWRN4uI&list=PLv_mO3iQ2o2fZllHFsEniuy1D-2IXr8eU Python в вопросах и ответах

Аудио: https://drive.google.com/file/d/1TLsB8EEMqATs18Iwacw5H3nceOFeO7ND/view?usp=sharing ООП в Пайтон

Текст: https://docs.google.com/document/d/1GwrAY4NoqPImiSMola-14ykVB5K-8iDlivWzvuEnMfA/edit?usp=sharing Объяснение алгоритмов сортировки с примерами на Python

P.S. аудио-файлы для обработки whisper стоит сохранить с именем на латинице и без пробелов в названии.

In [ ]:
# указываем ссылку для получения аудио-дорожки из видео
yt_urls = ['https://www.youtube.com/watch?v=LRuszWRN4uI&list=PLv_mO3iQ2o2fZllHFsEniuy1D-2IXr8eU']
YouTube_video_title = "Python in Questions and Answers"

In [ ]:
!pip install -q git+https://github.com/ytdl-org/youtube-dl.git
!pip install -q git+https://github.com/openai/whisper.git

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q tiktoken==0.4.0 openai==0.28.0 langchain==0.0.281 faiss-cpu==1.7.4
!pip install -q pydub
!pip install -qq python-docx
!pip install -U pytube
from IPython.display import clear_output
clear_output()

In [ ]:
import whisper
import os
import gdown
import requests
import re
import time
from IPython.display import HTML, clear_output
import subprocess
from pathlib import Path
import json
from pytube import YouTube
import tiktoken
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import ipywidgets as widgets
from IPython.display import display
from tqdm.auto import tqdm
import getpass
import pickle
from urllib.request import urlopen
import openai
import subprocess
import codecs
from langchain.chains import ConversationChain         # Импортируем класс для создания цепочек диалогов
from langchain.chat_models import ChatOpenAI           # Импортируем класс для работы с чатами на базе OpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory  # Импортируем класс для управления памятью диалогов
from langchain.text_splitter import MarkdownHeaderTextSplitter, Document, RecursiveCharacterTextSplitter
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import urllib
from pydub import AudioSegment
from google.colab import drive
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
# Получаем первую (и единственную) ссылку из списка yt_urls
url = yt_urls[0]

# Используем youtube-dl для получения имени файла, который будет сохранен
file_name = !youtube-dl $url -f 'bestaudio[ext=m4a]' --get-filename -o 'tmp/%(title)s.m4a'

# Загружаем аудио с лучшим качеством (в формате m4a)
!youtube-dl $url -f 'bestaudio[ext=m4a]' -o 'tmp/%(title)s.m4a'

/bin/bash: line 1: -f: command not found
[youtube] LRuszWRN4uI: Downloading webpage
[download] Изучаем Python _ 1 глава - 'Python в вопросах и ответах' с Алексеем Трояновым-LRuszWRN4uI.mkv has already been downloaded and merged


In [ ]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
# Переименуем файл
os.rename("Изучаем Python _ 1 глава - 'Python в вопросах и ответах' с Алексеем Трояновым-LRuszWRN4uI.mkv", 'Copy_Python_in_Questions_and_Answers.mkv')

In [ ]:
# Конвертируем файл из формата mkv в формат m4a
!ffmpeg -i '/content/Copy_Python_in_Questions_and_Answers.mkv' -vn -acodec copy '/content/Python_in_Questions_and_Answers.m4a'


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
def transcribe_audio_whisper_chunked(audio_path, file_title, save_folder_path, max_duration=5 * 60 * 1000):  # 5 минут
    """
    Транскрибирует аудиофайл по частям, чтобы соответствовать ограничениям размера API.
    """
    # Создаем каталог для сохранения результатов, если он не существует
    os.makedirs(save_folder_path, exist_ok=True)
    # Загружаем аудиофайл
    audio = AudioSegment.from_file(audio_path)
    # Создаем временную папку для хранения частей аудио
    temp_dir = os.path.join(save_folder_path, "temp_audio_chunks")
    os.makedirs(temp_dir, exist_ok=True)

    # Инициализируем переменные для обработки аудио по частям
    current_start_time = 0    # Текущее время начала фрагмента
    chunk_index = 1           # Индекс текущего фрагмента
    transcriptions = []       # Список для сохранения транскрибаций каждого фрагмента

    # Цикл по всему аудио
    while current_start_time < len(audio):
        # Вырезаем часть аудиофайла длительностью не более max_duration
        chunk = audio[current_start_time:current_start_time + max_duration]
        # Формируем имя файла для части аудио
        chunk_name = f"chunk_{chunk_index}.wav"
        # Путь к файлу части аудио
        chunk_path = os.path.join(temp_dir, chunk_name)
        # Экспортируем часть аудио в файл
        chunk.export(chunk_path, format="wav")

        # Проверяем размер файла перед отправкой
        if os.path.getsize(chunk_path) > 26214400:
            print(f"Chunk {chunk_index} exceeds the maximum size limit for the API. Trying a smaller duration...")
            max_duration = int(max_duration * 0.9)  # Попробуем уменьшить на 10% и попробовать снова
            os.remove(chunk_path)   # Удаляем слишком большой файл
            continue  # Пропускаем текущий кусок и пытаемся снова с меньшей длительностью

        # Процесс транскрибации фрагмента
        with open(chunk_path, "rb") as src_file:
            print(f"Transcribing {chunk_name}...")
            try:
                # Вызов API для транскрибации аудио
                transcription = openai.Audio.transcribe("whisper-1", src_file)
                # Добавляем результат транскрибации в список
                transcriptions.append(transcription["text"])
            except openai.error.APIError as e:
                print(f"An error occurred: {e}")
                break   # В случае ошибки API прерываем цикл

        # Удаляем обработанный фрагмент
        os.remove(chunk_path)
        # Переходим к следующему фрагменту аудио
        current_start_time += max_duration
        chunk_index += 1
    # Удаляем временную папку с фрагментами аудио
    os.rmdir(temp_dir)

    # Формируем путь к файлу, в который будет сохранена итоговая транскрибация
    result_path = os.path.join(save_folder_path, f"{file_title}.txt")
    with open(result_path, "w") as txt_file:
        # Сохраняем все транскрибации в один файл, разделяя их переносами строк
        txt_file.write("\n".join(transcriptions))
    # Выводим сообщение об успешном сохранении транскрибации
    print(f"Transcribe saved to {result_path}")

# Задаем исходные параметры и вызываем функцию
audio_path = '/content/Python_in_Questions_and_Answers.m4a'
file_title = 'Python_in_Questions_and_Answers'
save_folder_path = '/content'

# Вызов функции для транскрибации аудиофайла
transcribe_audio_whisper_chunked(audio_path, file_title, save_folder_path)

Chunk 1 exceeds the maximum size limit for the API. Trying a smaller duration...
Chunk 1 exceeds the maximum size limit for the API. Trying a smaller duration...
Chunk 1 exceeds the maximum size limit for the API. Trying a smaller duration...
Chunk 1 exceeds the maximum size limit for the API. Trying a smaller duration...
Chunk 1 exceeds the maximum size limit for the API. Trying a smaller duration...
Chunk 1 exceeds the maximum size limit for the API. Trying a smaller duration...
Chunk 1 exceeds the maximum size limit for the API. Trying a smaller duration...
Transcribing chunk_1.wav...
Transcribing chunk_2.wav...
Transcribing chunk_3.wav...
Transcribing chunk_4.wav...
Transcribing chunk_5.wav...
Transcribing chunk_6.wav...
Transcribing chunk_7.wav...
Transcribing chunk_8.wav...
Transcribing chunk_9.wav...
Transcribing chunk_10.wav...
Transcribing chunk_11.wav...
Transcribing chunk_12.wav...
Transcribing chunk_13.wav...
Transcribing chunk_14.wav...
Transcribing chunk_15.wav...
Transcr

In [ ]:
# Подключаемся к гугл диску
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
!whisper "/content/drive/MyDrive/OOP_in_Python.m4a" --model large --language Russian

100%|██████████████████████████████████████| 2.88G/2.88G [00:30<00:00, 102MiB/s]
[00:00.220 --> 00:09.980]  Хаудихо, друзья! В комментариях все уже давно просят сделать ролик про ООП и объяснить, что же это такое и для чего оно вообще нужно.
[00:10.260 --> 00:15.800]  Так что давайте познакомимся с, казалось бы, такой страшной штукой, как ООП.
[00:15.960 --> 00:26.900]  Хотя на самом деле в этом нет ничего сложного, потому что ООП было придумано лишь для того, чтобы облегчить жизнь программистам, но никак не наоборот.
[00:26.900 --> 00:34.940]  И если вы вспомните, как начинали учиться программировать, то вас скорее всего учили писать код в процедурном стиле.
[00:35.060 --> 00:44.660]  Это значит, что код, который вы писали, выполнялся последовательно и состоял преимущественно из всевозможных подпрограмм или же обычных функций.
[00:44.820 --> 00:49.820]  Так вот, до изобретения ООП таким образом код писали по сути все.
[00:49.820 --> 00:56.880]  И прикол тут в том, что пока ваша програ

In [ ]:
# Функция для загрузки документа по ссылке из Google Drive
def load_document_text(url: str) -> str:
    # Извлекаем идентификатор документа из URL-адреса
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Загружаем документ в виде обычного текста
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
sort_text= load_document_text('https://docs.google.com/document/d/1GwrAY4NoqPImiSMola-14ykVB5K-8iDlivWzvuEnMfA')

In [ ]:
sort_text[:1000]

'\ufeffОбъяснение алгоритмов сортировки с примерами на Python\r\n\r\n\r\nВ этой статье будут рассмотрены популярные алгоритмы, принципы их работы и реализация на Python. А ещё сравним, как быстро они сортируют элементы в списке.\r\nВ качестве общего примера возьмём сортировку чисел в порядке возрастания. Но эти методы можно легко адаптировать под ваши потребности.\r\nПузырьковая сортировка\r\nЭтот простой алгоритм выполняет итерации по списку, сравнивая элементы попарно и меняя их местами, пока более крупные элементы не «всплывут» в начало списка, а более мелкие не останутся на «дне».\r\nАлгоритм\r\nСначала сравниваются первые два элемента списка. Если первый элемент больше, они меняются местами. Если они уже в нужном порядке, оставляем их как есть. Затем переходим к следующей паре элементов, сравниваем их значения и меняем местами при необходимости. Этот процесс продолжается до последней пары элементов в списке.\r\nПри достижении конца списка процесс повторяется заново для каждого эле

In [ ]:
# Записываем в объяснение сортировки в текстовый файл
with open('Sorting_algorithms_in_Python.txt', 'w') as file:
    file.write(sort_text)

In [ ]:
# Создаем список с файлами для объединения
files_to_merge = ['Python_in_Questions_and_Answers.txt', 'OOP_in_Python.txt', 'Sorting_algorithms_in_Python.txt']

In [ ]:
# Объединяем и записываем текстовые файлы
output_file_name = 'merged_file.txt'

with open(output_file_name, 'w') as outfile:
    for fname in files_to_merge:
        with open(fname, 'r') as infile:
            outfile.write(infile.read())
            outfile.write("\n\n")

In [ ]:
#@title Напишите самостоятельно промпт в систем и юзер для разделения текста на логические блоки с выделением названия раздела:
system = '\u0412\u044B \u0433\u0435\u043D\u0438\u0439 \u0442\u0435\u043A\u0441\u0442\u0430, \u043A\u043E\u043F\u0438\u0440\u0430\u0439\u0442\u0438\u043D\u0433\u0430, \u043F\u0438\u0441\u0430\u0442\u0435\u043B\u044C\u0441\u0442\u0432\u0430. \u0412\u0430\u0448\u0430 \u0437\u0430\u0434\u0430\u0447\u0430 \u0440\u0430\u0441\u043F\u043E\u0437\u043D\u0430\u0442\u044C \u0440\u0430\u0437\u0434\u0435\u043B\u044B \u0432 \u0442\u0435\u043A\u0441\u0442\u0435 \u0438 \u0440\u0430\u0437\u0431\u0438\u0442\u044C \u0435\u0433\u043E \u043D\u0430 \u044D\u0442\u0438 \u0440\u0430\u0437\u0434\u0435\u043B\u044B \u0441\u043E\u0445\u0440\u0430\u043D\u044F\u044F \u0432\u0435\u0441\u044C \u0442\u0435\u043A\u0441\u0442 \u043D\u0430 100%' #@param {type:"string"}
user = "\u041F\u043E\u0436\u0430\u043B\u0443\u0439\u0441\u0442\u0430, \u0434\u0430\u0432\u0430\u0439\u0442\u0435 \u043F\u043E\u0434\u0443\u043C\u0430\u0435\u043C \u0448\u0430\u0433 \u0437\u0430 \u0448\u0430\u0433\u043E\u043C: \u041F\u043E\u0434\u0443\u043C\u0430\u0439\u0442\u0435, \u043A\u0430\u043A\u0438\u0435 \u0440\u0430\u0437\u0434\u0435\u043B\u044B \u0432 \u0442\u0435\u043A\u0441\u0442\u0435 \u0432\u044B \u043C\u043E\u0436\u0435\u0442\u0435 \u0440\u0430\u0441\u043F\u043E\u0437\u043D\u0430\u0442\u044C \u0438 \u043A\u0430\u043A\u043E\u0435 \u043D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u043F\u043E \u0441\u043C\u044B\u0441\u043B\u0443 \u043C\u043E\u0436\u043D\u043E \u0434\u0430\u0442\u044C \u043A\u0430\u0436\u0434\u043E\u043C\u0443 \u0440\u0430\u0437\u0434\u0435\u043B\u0443. \u0414\u0430\u043B\u0435\u0435 \u043D\u0430\u043F\u0438\u0448\u0438\u0442\u0435 \u043E\u0442\u0432\u0435\u0442 \u043F\u043E \u0432\u0441\u0435\u043C\u0443 \u043F\u0440\u0435\u0434\u044B\u0434\u0443\u0449\u0435\u043C\u0443 \u043E\u0442\u0432\u0435\u0442\u0443 \u0432 \u043F\u043E\u0440\u044F\u0434\u043A\u0435: ## \u041D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u0440\u0430\u0437\u0434\u0435\u043B\u0430, \u043F\u043E\u0441\u043B\u0435 \u0447\u0435\u0433\u043E \u0432\u0435\u0441\u044C \u0442\u0435\u043A\u0441\u0442, \u043E\u0442\u043D\u043E\u0441\u044F\u0449\u0438\u0439\u0441\u044F \u043A \u044D\u0442\u043E\u043C\u0443 \u0440\u0430\u0437\u0434\u0435\u043B\u0443. \u0422\u0435\u043A\u0441\u0442:" #@param {type:"string"}

temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}
chunk_size = 6000 #@param {type: "slider", min: 1000, max: 7000, step:500}

In [ ]:
# Функция настройки стиля для переноса текста в выводе ячеек
# для изменения стиля отображения текста, так чтобы предотвратить переполнение текста за границы ячейки вывода и обеспечить его перенос.
def set_text_wrap_css():
    css = '''
    <style>
    pre {
        white-space: pre-wrap;
    }
    </style>
    '''
    display(HTML(css))

get_ipython().events.register('pre_run_cell', set_text_wrap_css)

# Функция подсчета количества токенов
def num_tokens_from_messages(messages, model='gpt-3.5-turbo-0301'):
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding('cl100k_base')

    if model in ['gpt-3.5-turbo-0301', 'gpt-3.5-turbo-0613', 'gpt-3.5-turbo-16k', 'gpt-3.5-turbo', 'gpt-3.5-turbo-1106']:
        num_tokens = 0

        for message in messages:
            num_tokens += 4

            for key, value in message.items():
                num_tokens += len(encoding.encode(value))

                if key == 'name':
                    num_tokens -= 1

        num_tokens += 2
        return num_tokens

    else:
        raise NotImplementedError(f'''num_tokens_from_messages() is not presently implemented for model {model}.''')


# Функция дробления текста на чанки
def split_text(txt_file, chunk_size=chunk_size):
    source_chunks = []
    splitter = RecursiveCharacterTextSplitter(separators=['\n', '\n\n', '. '], chunk_size=chunk_size, chunk_overlap=0)

    for chunk in splitter.split_text(txt_file):
        source_chunks.append(Document(page_content=chunk, metadata={}))

    print(f'\n\nТекст разбит на {len(source_chunks)} чанков.')

    return source_chunks


# Функция получения ответа от модели
def answer_index(system, user, chunk, temp=temperature, model='gpt-3.5-turbo'):

    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user + f'{chunk}'}
    ]

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    # Вывод количества токенов отключен
    # print(f'\n====================\n\n{num_tokens_from_messages(messages)} токенов будет использовано на чанк\n\n')
    answer = completion.choices[0].message.content

    return answer


def process_one_file(file_path, system, user):
    with open(file_path, 'r') as txt_file:
        text = txt_file.read()
    source_chunks = split_text(text)
    processed_text = ''
    unprocessed_text = ''

    for chunk in source_chunks:
        attempt = 0

        while attempt < 3:
            try:
                answer = answer_index(system, user, chunk.page_content)
                break  # Успешно получили ответ, выходим из цикла попыток

            except Exception as e:
                attempt += 1  # Увеличиваем счетчик попыток
                print(f'\n\nПопытка {attempt} не удалась из-за ошибки: {str(e)}')
                time.sleep(10)  # Ожидаем перед следующей попыткой
                if attempt == 3:
                    answer = ''
                    print(f'\n\nОбработка элемента {chunk} не удалась после 3 попыток')
                    unprocessed_text += f'{chunk}\n\n'

        processed_text += f'{answer}\n\n'  # Добавляем ответ в результат
        print(f'{answer}')  # Выводим ответ

    return processed_text, unprocessed_text

In [ ]:
file_path = f'/content/merged_file.txt'
# Вызываем функцию обработки для этого файла
processed_text, unprocessed_text = process_one_file(file_path, system, user)



Текст разбит на 9 чанков.
## Раздел 1: Введение в серию "Читаем Луца"
Всем привет, добро пожаловать. Этим стулом ГАМС начинает новую серию полукресел, а мы этим видео начинаем новую серию видео. Она будет называться «Читаем Луца». Когда человек, который начинает учить питон, спрашивает, что же мне почитать, наверное, процентов 90 разработчиков «Читай Луца», там все есть. Это такой огромный двухтомник, он жутко нудный, он очень подробный, он достаточно плохо переведен, иногда он очень постноязычен. Но, к сожалению, пожалуй, лучше, чем этот учебник, просто по базовому питону пока ничего нет. Мы будем читать это дело по главам, на каждую главу у нас будет, скорее всего, отдельный оратор, и он будет давать презентацию, а по ходу вот этой презентации я буду задавать вопросы, каверзные и не очень, ну или подкидывать тему для обсуждения. Мы тут не одни, зрители, которые безмолвно присутствуют у нас сейчас в чате, они тоже будут задавать вопросы, мы их будем или засчитывать в конце, или мы б

In [ ]:
processed_text

'## Раздел 1: Введение в серию "Читаем Луца"\nВсем привет, добро пожаловать. Этим стулом ГАМС начинает новую серию полукресел, а мы этим видео начинаем новую серию видео. Она будет называться «Читаем Луца». Когда человек, который начинает учить питон, спрашивает, что же мне почитать, наверное, процентов 90 разработчиков «Читай Луца», там все есть. Это такой огромный двухтомник, он жутко нудный, он очень подробный, он достаточно плохо переведен, иногда он очень постноязычен. Но, к сожалению, пожалуй, лучше, чем этот учебник, просто по базовому питону пока ничего нет. Мы будем читать это дело по главам, на каждую главу у нас будет, скорее всего, отдельный оратор, и он будет давать презентацию, а по ходу вот этой презентации я буду задавать вопросы, каверзные и не очень, ну или подкидывать тему для обсуждения. Мы тут не одни, зрители, которые безмолвно присутствуют у нас сейчас в чате, они тоже будут задавать вопросы, мы их будем или засчитывать в конце, или мы будем в процессе обсуждения

In [ ]:
# Определяем заголовки, на которые следует разбить текст
headers_to_split_on = [
    ("##", "Header 2")
    ]
# Создаем объект для разбиения текста на секции по заголовкам
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Получаем список документов, разбитых по заголовкам
md_header_splits = markdown_splitter.split_text(processed_text)

In [ ]:
md_header_splits[:1]

[Document(page_content='Всем привет, добро пожаловать. Этим стулом ГАМС начинает новую серию полукресел, а мы этим видео начинаем новую серию видео. Она будет называться «Читаем Луца». Когда человек, который начинает учить питон, спрашивает, что же мне почитать, наверное, процентов 90 разработчиков «Читай Луца», там все есть. Это такой огромный двухтомник, он жутко нудный, он очень подробный, он достаточно плохо переведен, иногда он очень постноязычен. Но, к сожалению, пожалуй, лучше, чем этот учебник, просто по базовому питону пока ничего нет. Мы будем читать это дело по главам, на каждую главу у нас будет, скорее всего, отдельный оратор, и он будет давать презентацию, а по ходу вот этой презентации я буду задавать вопросы, каверзные и не очень, ну или подкидывать тему для обсуждения. Мы тут не одни, зрители, которые безмолвно присутствуют у нас сейчас в чате, они тоже будут задавать вопросы, мы их будем или засчитывать в конце, или мы будем в процессе обсуждения с ними как-то подключ

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(md_header_splits, embeddings)

In [ ]:
system_for_NA = "Вы преподаватель по языку Puthon. Ваша задача - ответить на вопрос студента только на основе представленных вам документов, не добавляя ничего из собственных знаний."

In [ ]:
def answer_neuro_assist(system, topic, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=3)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nExcerpt of document №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system_for_NA},
        {"role": "user", "content": f"Answer the student's question. Answer in Russian. Do not mention the excerpts of the student response information document in your answer. Student response document: {message_content}\n\nStudent Question: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

In [ ]:
# Проверяем работу
topic="Что такое ООП?"
ans=answer_neuro_assist(system, topic, db, verbose=1)
ans


 ===========================================: 
message_content :
 
Excerpt of document №1
=====================В этом разделе рассматривается понятие объектно-ориентированного программирования (ООП) и его основные принципы. Описывается, что такое классы, объекты, атрибуты и методы. Приводится пример создания класса color и объясняется, как работает инициализация объекта.  
Excerpt of document №2
=====================Все это позволяет создавать очень большие программы, над которыми могут работать сразу сотни или десятки тысяч программистов. Но при этом все они могут не знать, как устроена вся программа целиком. Потому что каждый из них будет работать лишь над ее отдельными частями. Ну и третий мощный инструмент ООП, который пока что можно пропустить, это полиморфизм. Потому что понимается он гораздо лучше тогда, когда вы уже изучите концепции ООП на практике. Пока что можете запомнить лишь то, что полиморфизм это когда функция способна обработать данные разных типов. Ну и по сути это в

'ООП (объектно-ориентированное программирование) - это подход к разработке программного обеспечения, основанный на концепции объектов. В ООП программа представляет собой набор взаимодействующих объектов, каждый из которых имеет свои собственные данные и функциональность. Объекты взаимодействуют друг с другом, обмениваясь сообщениями и вызывая методы других объектов. ООП позволяет создавать модульные и масштабируемые программы, где каждый объект выполняет свою задачу, но при этом является частью более крупной системы. Основные принципы ООП включают инкапсуляцию, наследование и полиморфизм.'

In [ ]:
# Проверяем работу
topic="Какая самая медленная сортировка?"
ans=answer_neuro_assist(system, topic, db, verbose=1)
ans


 ===========================================: 
message_content :
 
Excerpt of document №1
=====================Пузырьковая сортировка — самый медленный из всех алгоритмов. Возможно, он будет полезен как введение в тему алгоритмов сортировки, но не подходит для практического использования.  
Excerpt of document №2
=====================Сортировка вставками выполняет меньше сравнений, чем сортировка выборкой и в реальности должна быть производительнее, но в данном эксперименте она выполняется немного медленней. Сортировка вставками делает гораздо больше обменов элементами. Если эти обмены занимают намного больше времени, чем сравнение самих элементов, то такой результат вполне закономерен.  
Excerpt of document №3
=====================Быстрая сортировка хорошо оправдывает своё название, почти в два раза быстрее, чем сортировка слиянием, и не требуется дополнительное место для результирующего массива.


 ===========================================: 


'Самая медленная сортировка - пузырьковая сортировка.'

In [ ]:
#@title Обрабатываем каждый чанк, выделяя только суть для методички
system = "\u0412\u044B - \u0433\u0435\u043D\u0438\u0439 \u043A\u043E\u043F\u0438\u0440\u0430\u0439\u0442\u0438\u043D\u0433\u0430, \u044D\u043A\u0441\u043F\u0435\u0440\u0442 \u0432 \u043E\u0431\u043B\u0430\u0441\u0442\u0438 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F \u043D\u0430 Python. \u0412\u044B \u043F\u043E\u043B\u0443\u0447\u0430\u0435\u0442\u0435 \u0443\u0447\u0430\u0441\u0442\u043E\u043A \u043D\u0435\u043E\u0431\u0440\u0430\u0431\u043E\u0442\u0430\u043D\u043D\u043E\u0433\u043E \u0442\u0435\u043A\u0441\u0442\u0430 \u043D\u0430 \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u043D\u0443\u044E \u0442\u0435\u043C\u0443. \u0412\u0430\u043C \u043D\u0443\u0436\u043D\u043E \u0438\u0437\u0432\u043B\u0435\u0447\u044C \u0438\u0437 \u044D\u0442\u043E\u0433\u043E \u0442\u0435\u043A\u0441\u0442\u0430 \u0441\u0443\u0442\u044C, \u0442\u043E\u043B\u044C\u043A\u043E \u0441\u0430\u043C\u043E\u0435 \u0432\u0430\u0436\u043D\u043E\u0435, \u0441\u043E\u0445\u0440\u0430\u043D\u0438\u0432 \u0432\u0441\u0435 \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u044B\u0435 \u0434\u0435\u0442\u0430\u043B\u0438, \u043D\u043E \u0443\u0431\u0440\u0430\u0432 \u0432\u0441\u044E \"\u0432\u043E\u0434\u0443\" \u0438 \u0441\u043B\u043E\u0432\u0430 (\u043F\u0440\u0435\u0434\u043B\u043E\u0436\u0435\u043D\u0438\u044F), \u043D\u0435 \u043D\u0435\u0441\u0443\u0449\u0438\u0435 \u0441\u043C\u044B\u0441\u043B\u043E\u0432\u043E\u0439 \u043D\u0430\u0433\u0440\u0443\u0437\u043A\u0438." #@param {type:"string"}
user = "\u0418\u0437 \u044D\u0442\u043E\u0433\u043E \u0442\u0435\u043A\u0441\u0442\u0430 \u0432\u044B\u0431\u0435\u0440\u0438\u0442\u0435 \u0442\u043E\u043B\u044C\u043A\u043E \u0442\u0443 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044E, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442 \u0446\u0435\u043D\u043D\u043E\u0441\u0442\u044C \u0441 \u0442\u043E\u0447\u043A\u0438 \u0437\u0440\u0435\u043D\u0438\u044F \u0442\u0435\u043C\u044B \"\u041E\u0431\u0443\u0447\u0435\u043D\u0438\u044F \u044F\u0437\u044B\u043A\u0443 Python\". \u0423\u0434\u0430\u043B\u0438\u0442\u0435 \u0432\u0441\u044E \"\u0432\u043E\u0434\u0443\". \u0412 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0435 \u0443 \u0432\u0430\u0441 \u0434\u043E\u043B\u0436\u0435\u043D \u043F\u043E\u043B\u0443\u0447\u0438\u0442\u044C\u0441\u044F \u0440\u0430\u0437\u0434\u0435\u043B \u0434\u043B\u044F \u0440\u0443\u043A\u043E\u0432\u043E\u0434\u0441\u0442\u0432\u0430 \u043F\u043E \u044D\u0442\u043E\u0439 \u0442\u0435\u043C\u0435. \u041E\u0441\u043D\u043E\u0432\u044B\u0432\u0430\u0439\u0442\u0435\u0441\u044C \u0442\u043E\u043B\u044C\u043A\u043E \u043D\u0430 \u043F\u0440\u0438\u0432\u0435\u0434\u0435\u043D\u043D\u043E\u043C \u0442\u0435\u043A\u0441\u0442\u0435, \u043D\u0435 \u043F\u0440\u0438\u0434\u0443\u043C\u044B\u0432\u0430\u0439\u0442\u0435 \u043D\u0438\u0447\u0435\u0433\u043E \u0441\u0432\u043E\u0435\u0433\u043E. \u041E\u0442\u0432\u0435\u0442 \u0434\u043E\u043B\u0436\u0435\u043D \u0431\u044B\u0442\u044C \u0432 \u0444\u043E\u0440\u043C\u0430\u0442\u0435 ## \u041D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u0442\u0435\u043A\u0441\u0442\u0430, \u0430 \u0437\u0430\u0442\u0435\u043C \u0446\u0435\u043D\u043D\u0430\u044F \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044F \u0438\u0437 \u0442\u0435\u043A\u0441\u0442\u0430. \u0415\u0441\u043B\u0438 \u0432 \u0442\u0435\u043A\u0441\u0442\u0435 \u043D\u0435\u0442 \u0446\u0435\u043D\u043D\u043E\u0439 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u0438, \u043E\u0441\u0442\u0430\u0432\u044C\u0442\u0435 \u0442\u043E\u043B\u044C\u043A\u043E \u043D\u0430\u0437\u0432\u0430\u043D\u0438\u0435 \u0440\u0430\u0437\u0434\u0435\u043B\u0430, \u043D\u0430\u043F\u0440\u0438\u043C\u0435\u0440: \"## \u0412\u0432\u0435\u0434\u0435\u043D\u0438\u0435\". \u0422\u0435\u043A\u0441\u0442:  \u041F\u0435\u0440\u0435\u0432\u0435\u0434\u0435\u043D\u043E \u0441 \u043F\u043E\u043C\u043E\u0449\u044C\u044E www.DeepL.com/Translator (\u0431\u0435\u0441\u043F\u043B\u0430\u0442\u043D\u0430\u044F \u0432\u0435\u0440\u0441\u0438\u044F)" #@param {type:"string"}

temperature = 0 #@param {type: "slider", min: 0, max: 1, step:0.1}


In [ ]:
def process_documents(documents, system, user, temperature):
    """
    Функция принимает чанки, system, user, temperature для модели.
    Она обрабатывает каждый документ, используя модель GPT, конкатенирует результаты в один текст и сохраняет в файл .txt.
    """
    processed_text_for_handbook = ""  # Строка для конкатенации обработанного текста

    for document in documents:
        # Форматируем метаданные для включения в чанк
        metadata_str = "\n".join([f"{key}: {value}" for key, value in document.metadata.items()])
        # Конкатенируем метаданные и содержание документа для передачи в функцию
        chunk_with_metadata = f"{metadata_str}\n\n{document.page_content}"

        # Получаем ответ от модели
        answer = answer_index(system, user, chunk_with_metadata, temperature, model='gpt-3.5-turbo')
        # Добавляем обработанный текст в общую строку
        processed_text_for_handbook += f"{answer}\n\n"

    # Записываем полученный текст в файл
    with open('processed_documents.txt', 'w', encoding='utf-8') as f:
        f.write(processed_text_for_handbook)

    # Функция возвращает путь к файлу с обработанным текстом
    return 'processed_documents.txt'

# Применение функции
file_path = process_documents(md_header_splits, system, user, temperature)
print(f"Обработанный текст сохранен в файле: {file_path}")

Обработанный текст сохранен в файле: processed_documents.txt


In [ ]:
# Содержимое методички:
file_path = '/content/processed_documents.txt'

with open(file_path, 'r') as file:
    content = file.read()
    print(content)

## Введение в серию "Читаем Луца"
- Серия видео "Читаем Луца" предназначена для тех, кто начинает изучать язык Python.
- Книга Марка Лутца "Изучаем Python" является рекомендуемым источником для изучения языка Python.
- Книга является подробным и достаточно объемным руководством.
- В серии будут рассмотрены главы книги, каждая глава будет представлена отдельным оратором.
- В ходе презентации будут задаваться вопросы и обсуждаться темы с зрителями.
- Сегодня будет обсуждаться первая глава книги "Изучаем Python".

## Особенности книги "Изучаем Python"
- Книга "Изучаем Python" имеет 5-е издание на английском языке и вышла на русском языке в 2019 году.
- Она рассчитана на версию языка Python 3.
- В книге рассматриваются различные фичи языка, такие как хинты типов, дата классы, морж оператор и другие.
- Важно учитывать, что некоторые фичи могут быть добавлены в более новых версиях Python, поэтому граничные условия нужно иметь в виду при чтении книги.

## Почему выбирают Python

- Python обла